<center><h1><b>No SQL Data Modeling Using Cassandra and Python</b></h1></center>

![cassandra](https://miro.medium.com/max/700/1*xlRpJixAInrMlIhn5KINgA.jpeg)

# *1. ETL Pipeline for PreProcessing Data*

# *Run the following code to Preprocess the file*

In [56]:
# import the required package
import pandas as pd
import os
import glob
import csv

## value is list of data that we have to fetch from event_data

In [15]:
value=['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'] 

In [59]:
# This will give us the current working directory and we will add then ouer required directory path ie. 'event_data'
path = os.getcwd() + '/event_data'
print(path)

/home/devil/Desktop/work/data_enginerr/data_modeling/event_data


In [60]:
# this code will extract all the file path inside given directory ending with '.csv'
file_path_list = []
for file in glob.glob(path+'/*.csv'):
    file_path_list.append(file)
    print(file)

/home/devil/Desktop/work/data_enginerr/data_modeling/event_data/2018-11-06-events.csv
/home/devil/Desktop/work/data_enginerr/data_modeling/event_data/2018-11-29-events.csv
/home/devil/Desktop/work/data_enginerr/data_modeling/event_data/2018-11-23-events.csv
/home/devil/Desktop/work/data_enginerr/data_modeling/event_data/2018-11-30-events.csv
/home/devil/Desktop/work/data_enginerr/data_modeling/event_data/2018-11-17-events.csv
/home/devil/Desktop/work/data_enginerr/data_modeling/event_data/2018-11-24-events.csv
/home/devil/Desktop/work/data_enginerr/data_modeling/event_data/2018-11-02-events.csv
/home/devil/Desktop/work/data_enginerr/data_modeling/event_data/2018-11-26-events.csv
/home/devil/Desktop/work/data_enginerr/data_modeling/event_data/2018-11-28-events.csv
/home/devil/Desktop/work/data_enginerr/data_modeling/event_data/2018-11-19-events.csv
/home/devil/Desktop/work/data_enginerr/data_modeling/event_data/2018-11-20-events.csv
/home/devil/Desktop/work/data_enginerr/data_modeling/e

In [72]:
# initiating an empty list of rows that will be generated from each file
full_data_row = []

# for every filepath in the file path list 
for f in file_path_list:
    
# reading csv file 
    with open(f,'r',encoding='utf',newline='') as csv_file:
        csvreader = csv.reader(csv_file)
        next(csvreader)
        
# extracting each data row one by one and append it 
        for line in csvreader:
            full_data_row.append(line)
# print full data row
# print(full_data_row)

# creating a smaller event data csv file called new_event_data.csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect',quoting=csv.QUOTE_ALL,skipinitialspace=True)
with open('new_event_data.csv','w',encoding='utf',newline='') as f:
    writer = csv.writer(f,dialect='myDialect')
    writer.writerow(value)
    for row in full_data_row:
        if row[0]=='':
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))

# *Part 2.  Apache Cassandra coding portion of our project* 

## *Now you are ready to work with the CSV file titled <font color=red>new_event_data.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns*: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**new_event_data.csv**</font> after the code above is run:<br>

<img src="image/download.png">

In [64]:
import cassandra
from cassandra.cluster import Cluster

In [73]:
# This should make a connection to a Cassandra instance our local machine 
# (127.0.0.1)
try:
    cluster = Cluster(['127.0.0.1'])
    session = cluster.connect()
except Exception as e:
    print(f'Error is : {e}')

In [78]:
# TO-DO: Create a Keyspace
try:
    session.execute('''
CREATE KEYSPACE IF NOT EXISTS sparkifydb
WITH REPLICATION = 
{'class':'SimpleStrategy','replication_factor':1}
''')
except Exception as e:
    print(f'Error is: {e}')

In [80]:
# TO-DO: Set KEYSPACE to the keyspace specified above

try:
    session.set_keyspace('sparkifydb')
except Exception as e:
    print(f'Error is: {e}')

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [101]:
## TO-DO: Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4
query_1 = 'CREATE TABLE IF NOT EXISTS music_history'
query_1 = query_1 + '(session_id INT,item_session INT, song_title TEXT, artist_name TEXT, song_length FLOAT, PRIMARY KEY(session_id,item_session))'
try:
    session.execute(query_1)
except Exception as e:
    print(f'Error is: {e}')

In [102]:
df = pd.read_csv('new_event_data.csv')
df.head()

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
0,Gustavo Cerati,Adler,M,1,Barrera,249.44281,free,"New York-Newark-Jersey City, NY-NJ-PA",248,Uno Entre 1000,100
1,Limp Bizkit,Adler,M,2,Barrera,270.49751,free,"New York-Newark-Jersey City, NY-NJ-PA",248,Behind Blue Eyes,100
2,Mikel Erentxun,Samuel,M,1,Gonzalez,178.83383,free,"Houston-The Woodlands-Sugar Land, TX",252,Frases Mudas,61
3,The Gerbils,Martin,M,0,Johnson,27.01016,free,"Minneapolis-St. Paul-Bloomington, MN-WI",250,(iii),55
4,AFI,Martin,M,1,Johnson,190.45832,free,"Minneapolis-St. Paul-Bloomington, MN-WI",250,Girl's Not Grey,55


In [95]:
# get the data

len_=len(df['artist'])
for i in range(len_):
    print(df.iloc[i][8],df.iloc[i][3],df.iloc[i][9],df.iloc[i][0],df.iloc[i][5])

In [106]:
# Insert data for first gtable music_history
query_1 = "INSERT INTO music_history(session_id,item_session,song_title,artist_name,song_length)"
query_1 = query_1 + 'VALUES(%s,%s,%s,%s,%s)'
try:
    len_=len(df['artist'])
    for i in range(len_):
        session.execute(query_1,(df.iloc[i][8],df.iloc[i][3],df.iloc[i][9],df.iloc[i][0],df.iloc[i][5]))
except Exception as e:
    print(f'Error is: {e}')

In [109]:
# Now validate your insert after runing some query:
try:
    rows=session.execute('SELECT artist_name,song_title,song_length FROM music_history WHERE session_id = 338 AND item_session = 4')
    for row in rows:
        print(row)
except Exception as e:
    print(f'Error is: {e}')

Row(artist_name='Faithless', song_title='Music Matters (Mark Knight Dub)', song_length=495.30731201171875)


In [110]:
## TO-DO: Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182
query_2 = 'CREATE TABLE IF NOT EXISTS user_choice'
query_2 = query_2 + '(user_id INT,item_session INT, session_id INT, first_name TEXT, last_name TEXT, artist_name TEXT,song_name TEXT, PRIMARY KEY(user_id,session_id,item_session))'
try:
    session.execute(query_2)
except Exception as e:
    print(f'Error is: {e}')

In [111]:
df = pd.read_csv('new_event_data.csv')
df.head()

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
0,Gustavo Cerati,Adler,M,1,Barrera,249.44281,free,"New York-Newark-Jersey City, NY-NJ-PA",248,Uno Entre 1000,100
1,Limp Bizkit,Adler,M,2,Barrera,270.49751,free,"New York-Newark-Jersey City, NY-NJ-PA",248,Behind Blue Eyes,100
2,Mikel Erentxun,Samuel,M,1,Gonzalez,178.83383,free,"Houston-The Woodlands-Sugar Land, TX",252,Frases Mudas,61
3,The Gerbils,Martin,M,0,Johnson,27.01016,free,"Minneapolis-St. Paul-Bloomington, MN-WI",250,(iii),55
4,AFI,Martin,M,1,Johnson,190.45832,free,"Minneapolis-St. Paul-Bloomington, MN-WI",250,Girl's Not Grey,55


In [114]:
# Insert data for first gtable music_history
query_2 = "INSERT INTO user_choice(user_id,item_session,session_id,first_name,last_name,artist_name,song_name)"
query_2 = query_2 + 'VALUES(%s,%s,%s,%s,%s,%s,%s)'
try:
    len_=len(df['artist'])
    for i in range(len_):
        session.execute(query_2,(df.iloc[i][10],df.iloc[i][3],df.iloc[i][8],df.iloc[i][1],df.iloc[i][4],df.iloc[i][0],df.iloc[i][9]))
except Exception as e:
    print(f'Error is: {e}')

In [115]:
# Now validate your insert after runing some query:
try:
    rows=session.execute('SELECT artist_name,song_name,first_name,last_name FROM user_choice WHERE user_id = 10 AND session_id = 182')
    for row in rows:
        print(row)
except Exception as e:
    print(f'Error is: {e}')

Row(artist_name='Down To The Bone', song_name="Keep On Keepin' On", first_name='Sylvie', last_name='Cruz')
Row(artist_name='Three Drives', song_name='Greece 2000', first_name='Sylvie', last_name='Cruz')
Row(artist_name='Sebastien Tellier', song_name='Kilometer', first_name='Sylvie', last_name='Cruz')
Row(artist_name='Lonnie Gordon', song_name='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', first_name='Sylvie', last_name='Cruz')


In [125]:
## TO-DO: Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4
query_3 = 'CREATE TABLE IF NOT EXISTS music_history_app'
query_3 = query_3 + '(session_id INT,item_session INT, song_title TEXT, artist_name TEXT, song_length FLOAT,first_name TEXT,last_name TEXT, PRIMARY KEY(song_title))'
try:
    session.execute(query_3)
except Exception as e:
    print(f'Error is: {e}')

In [126]:
# Insert data for first gtable music_history
query_3 = "INSERT INTO music_history_app(session_id,item_session,song_title,artist_name,song_length,first_name,last_name)"
query_3 = query_3 + 'VALUES(%s,%s,%s,%s,%s,%s,%s)'
try:
    len_=len(df['artist'])
    for i in range(len_):
        session.execute(query_3,(df.iloc[i][8],df.iloc[i][3],df.iloc[i][9],df.iloc[i][0],df.iloc[i][5],df.iloc[i][1],df.iloc[i][4]))
except Exception as e:
    print(f'Error is: {e}')

In [127]:
# Now validate your insert after runing some query:
try:
    rows=session.execute("SELECT artist_name,song_title,song_length,first_name,last_name FROM music_history_app WHERE song_title='All Hands Against His Own'")
    for row in rows:
        print(row)
except Exception as e:
    print(f'Error is: {e}')

Row(artist_name='The Black Keys', song_title='All Hands Against His Own', song_length=196.9105682373047, first_name='Tegan', last_name='Levine')


In [128]:
session.shutdown()
cluster.shutdown()